# Dataset sizes

Statistics on the datasets used for training and testing

In [2]:
import datasets
import pandas as pd 

from transnormer.data import loader

In [7]:

paths = [
    "../../data/raw/ridges/bollmann-split/ridges.dev.txt",
    "../../data/raw/ridges/bollmann-split/ridges.test.txt",
    "../../data/raw/ridges/bollmann-split/ridges.train.txt",
    "../../data/raw/dta/dtaeval/split-v3.0/xml/dev",
    "../../data/raw/dta/dtaeval/split-v3.0/xml/test",
    "../../data/raw/dta/dtaeval/split-v3.0/xml/train",
    "../../data/raw/leipzig-corpora/deu_news_2020_1M-sentences.txt",
    ]


df_data = {"name" : [], "split" : [], "examples" : [] }
for name, split, data in loader.load_data(paths):
    ds = datasets.Dataset.from_dict(data)
    df_data["name"].append(name)
    df_data["split"].append(split)
    df_data["examples"].append(ds.num_rows)
df = pd.DataFrame(data=df_data)


Line 8257 does not have length 2 but 3 skip line: '.	.I.	I.'
Line 10671 does not have length 2 but 3 skip line: '.	.Vorred	Vorrede'
Line 16413 does not have length 2 but 1 skip line: 'dañ'


In [8]:
df

,name,split,examples
0,ridges_bollmann,validation,671
1,ridges_bollmann,test,671
2,ridges_bollmann,train,2921
3,dtaeval,validation,18278
4,dtaeval,test,21916
5,dtaeval,train,200524
6,deu_news_2020,,1000000
